In [1]:
import geopandas as gpd

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [5]:
gsap_2021_df = gpd.read_file('../data/GSAP_geom/GSAP_AM24_2021.shp')

In [6]:
gsap_2021_df.keys()

Index(['code', 'geo_year', 'geo_source', 'geo_level', 'geo_idvar', 'geo_id',
       'geo_nvar', 'geo_name', 'geo_code', 'geometry'],
      dtype='object')

In [28]:
for group, data in gsap_2021_df.groupby('code'):
    base = '../app/public/data/sub-national-geometry/'
    data = data[['geo_code','geo_name','geometry']].copy()
    data['geometry'] = data['geometry'].simplify(tolerance=0.001, preserve_topology=True)
    data.to_file(os.path.join(base,f"{group}_gsap_geometry.geojson"),driver='GeoJSON')